# IHME PROJECTIONS

IHME's COVID-19 projections were developed in response to requests from the University of Washington School of Medicine and other US hospital systems and state governments working to determine when COVID-19 would overwhelm their ability to care for patients. The forecasts show demand for hospital services, including the availability of ventilators, general hospital beds, and ICU beds, as well as daily and cumulative deaths due to COVID-19.

In [ ]:
import pandas as pd
import requests
import pycountry
import zipfile
from functools import reduce
import io
import re
from datetime import datetime
from csv import QUOTE_NONNUMERIC

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
# get .zip file content

response = requests.get("https://ihmecovid19storage.blob.core.windows.net/latest/ihme-covid19.zip", stream=True)
assert response.status_code is 200

# parse .zip
z = zipfile.ZipFile(io.BytesIO(response.content))

In [ ]:
# create df from csv in zip

df = pd.read_csv(z.open(re.search(r".*\.csv", "\n".join(z.namelist()))[0]))

In [ ]:
# set columns Last_Update_Date and LAST_REPORTED_FLAG

df['Last_Update_Date'] = datetime.now()
df['Last_Reported_Flag'] = df['date'].max() == df['date']

In [ ]:
regions_manual_dict = {
    "Balearic Islands": ("ES", "Spain", "ES-IB"),
    "Basque Country": ("ES", "Spain", "ES-PV"),
    "Bavaria": ("DE", "Germany", "DE-BY"),
    "Canary Islands": ("ES", "Spain", "ES-CN"),
    "Castile and Leon": ("ES", "Spain", "ES-CL"),
    "Catalonia": ("ES", "Spain", "ES-CT"),
    "Community of Madrid": ("ES", "Spain", "ES-MD"),
    "King and Snohomish Counties (excluding Life Care Center), WA": ("US", "United States", "US-WA"),
    "Life Care Center, Kirkland, WA": ("US", "United States", "US-WA"),
    "Lower Saxony": ("DE", "Germany", "DE-NI"),
    "Navarre": ("ES", "Spain", "ES-NA"),
    "North Rhine-Westphalia": ("DE", "Germany", "DE-NW"),
    "Other Counties, WA": ("US", "United States", "US-WA"),
    "Provincia autonoma di Bolzano": ("IT", "Italy", "IT-BZ"),
    "Provincia autonoma di Trento": ("IT", "Italy", "IT-TN"),
    "Rhineland-Palatinate": ("DE", "Germany", "DE-RP"),
    "Saxony-Anhalt": ("DE", "Germany", "DE-ST"),
    "Saxony": ("DE", "Germany", "DE-SN"),
    "Thuringia": ("DE", "Germany", "DE-TH"),
    "Valencian Community": ("ES", "Spain", "ES-VC")
}

subdivisions = {}

In [ ]:
# create country resolver helper funcs
def resolve_country(row):
    location_name = row['location_name']
    lookup = pycountry.countries.get(name=location_name)
    if lookup:
        row['COUNTRY_REGION'] = lookup.name
        row['ISO_3166_1'] = lookup.alpha_2
        row['ISO_3166_2'] = None
    else:
        row['COUNTRY_REGION'] = None
        row['ISO_3166_1'] = None
        row['ISO_3166_2'] = None
    return row
        

def resolve_country_fuzzy(row):
    location_name = row['location_name']
    try:
        lookup = pycountry.countries.search_fuzzy(location_name)[0]  # lookup location, select best match
        row['COUNTRY_REGION'] = lookup.name
        country_code = lookup.alpha_2
        row['ISO_3166_1'] = country_code
        if country_code not in list(subdivisions):
            subdivisions[country_code] = {k.name: k.code.replace(f"{country_code}-", "") for k in pycountry.subdivisions.get(country_code=country_code)}
        row['ISO_3166_2'] = subdivisions[country_code][location_name]
        return row
    except (LookupError, TypeError, KeyError):
        return row
    
def resolve_manual(row):
    location_name = row['location_name']
    if location_name not in list(regions_manual_dict):
        return row
    country_code, country_name, subdiv_code = regions_manual_dict[location_name]
    row['COUNTRY_REGION'] = country_name
    row['ISO_3166_1'] = country_code
    row['ISO_3166_2'] = subdiv_code.replace(f"{country_code}-", "")
    return row

In [ ]:
df = df.apply(lambda row: resolve_country(row), axis=1)

In [ ]:
df = df.drop('V1', axis=1)
df

In [ ]:
df.loc[df['ISO_3166_1'].isna()] = df.loc[df['ISO_3166_1'].isna()].apply(lambda row: resolve_country_fuzzy(row), axis=1)

In [ ]:
df.loc[df['ISO_3166_1'].isna()] = df.loc[df['ISO_3166_1'].isna()].apply(lambda row: resolve_manual(row), axis=1)

In [ ]:
# fix some subdivisions manually

df['ISO_3166_2'].loc[(df['ISO_3166_2'].isna()) & (df['location_name'] == 'Aragon')] = "AR"
df['ISO_3166_2'].loc[(df['ISO_3166_2'].isna()) & (df['location_name'] == 'Andalucia')] = "AN"
df['ISO_3166_2'].loc[(df['ISO_3166_2'].isna()) & (df['location_name'] == 'Baden-Wurttemberg')] = "BW"
df['ISO_3166_2'].loc[(df['ISO_3166_2'].isna()) & (df['location_name'] == 'Hesse')] = "HE"

In [ ]:
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y-%m-%d')

In [ ]:
df['PROVINCE_STATE'] = None
df['PROVINCE_STATE'].loc[df['ISO_3166_2'].notna()] = df.loc[df['ISO_3166_2'].notna()]['location_name']

In [ ]:
df = df.drop(columns=['location_name'])

In [ ]:
df.dtypes

In [ ]:
df.sample(5)

In [ ]:
df.to_csv(output_folder + "IHME_COVID_19.csv", columns=["date","allbed_mean","allbed_lower","allbed_upper","ICUbed_mean","ICUbed_lower","ICUbed_upper","InvVen_mean","InvVen_lower","InvVen_upper","deaths_mean","deaths_lower","deaths_upper","admis_mean","admis_lower","admis_upper","newICU_mean","newICU_lower","newICU_upper","totdea_mean","totdea_lower","totdea_upper","bedover_mean","bedover_lower","bedover_upper","icuover_mean","icuover_lower","icuover_upper","Last_Update_Date","Last_Reported_Flag","COUNTRY_REGION","ISO_3166_1","ISO_3166_2","PROVINCE_STATE"],
          index=False, sep=",", quoting=QUOTE_NONNUMERIC)